# Introduction

## What is an Iteration?

> A process that is repeated more than one time by applying the same logic is called an Iteration.

In [43]:
a = [0, 5, 10, 15, 20]
for i in a:
    if i % 2 == 0:
        print(str(i)+' is an Even Number')
    else:
        print(str(i)+' is an Odd Number')

0 is an Even Number
5 is an Odd Number
10 is an Even Number
15 is an Odd Number
20 is an Even Number


## Problems before Iterators:

<ol>
    <li>
    Complex of Iteration Logic
        <ul>
            <li>Manually managing indices</li>
            <li>Hard to keep track of state while looping</li>
        </ul>
    </li>
    <li>
    Inefficient Memory Usage
        <ul>
            <li>Infinite sequences was a nightmare</li>
        </ul>
    </li>
    <li>
    Rigid and Repetitive Code
        <ul>
            <li>Hard to reusing iteration logic across different projects</li>
        </ul>
    </li>
    <li>
    Limited Flexibility for Complex Iterations
        <ul>
            <li>Filtering</li>
            <li>Skipping</li>
            <li>Generating dynamic sequences</li>
        </ul>
    </li>
</ol>

## Iterators

An iterator is an <span style='color: green'>object</span> which <span style='color: green'>contains a countable number of values</span> and it is used to <span style='color: yellow'>iterate over iterable objects like list, tuples, sets, etc.</span> Iterators are <span style='color: green'>implemented using </span> <span style='color: yellow'>a class</span> and a local variable for iterating is not required
<br>
<br>
It follows <span style='color: yellow'>Lazy Evaluation</span> where the evaluation of the expression will be on hold and stored in the memory until the item is called specifically which helps us to avoid repeated evaluation. As lazy evaluation is implemented, it requires only 1 memory location to process the value and when we are using a large dataset then, wastage of RAM space will be reduced the need to load the entire dataset at the same time will not be there.

- <strong style='color: yellow'>iter()</strong> function is used to create an iterator containing an iterable object.
- <strong style='color: yellow'>next()</strong> function is used to call the next element in the iterable object.
- Once an iterator is exhausted, it cannot be reused. After the iterable object is completed, to use them again reassign them to the same object.

In [44]:
# iter() / .__iter__()
# next() / .__next__()

iter_list = iter(['Ostad', 'Salehi', 'is', 'The Best'])
# iter_list = ['Ostad', 'Salehi', 'is', 'The Best'].__iter__()

# next()
print(next(iter_list))
# .__next__()
print(iter_list.__next__())


print(next(iter_list))
print(next(iter_list))

Ostad
Salehi
is
The Best


### StopIteration Exception:

In [45]:
print(next(iter_list))

StopIteration: 

### Memory Usage

In [ ]:
import sys

x = [1, 2, 3, 4, 5]

print('List size (1 to 5): ', sys.getsizeof(x), 'bytes')
print('Iterator size  (1 to 5): ', sys.getsizeof(iter(x)), 'bytes')

### Make Iterator using class (old syntax)

In [ ]:
class ExampleIterator:
    def __init__(self, n: int) -> None:
        self.n = n

    def __iter__(self) -> object:
        self.current = 0
        return self
    
    def __next__(self):
        self.current += 1
        if self.current > self.n:
            raise StopIteration
        
        return self.current

## Generators

Generators are another newer way of creating iterators in a simple way where it uses the keyword “yield” instead of returning it in a defined function.

Generators are implemented using a function.

Elegant way of creating ``ExampleIter`` :

In [39]:
def example_generator(n):
    for i in range(1, n+1):
        yield i # pause and not terminating the function

In [ ]:
print("The square of numbers are : ")
for i in example_generator(5):
    print(i)

## Table of Difference Between Iterator vs Generators

| **Iterator**                                                                 | **Generator**                                                                                   |
|------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------|
| Class is used to implement an iterator                                       | Function is used to implement a generator.                                                     |
| Local Variables aren’t used here.                                            | All the local variables before the `yield` function are stored.                                  |
| Iterator uses `iter()` and `next()` functions                                | Generator uses `yield` keyword                                                                 |
| Every iterator is not a generator                                            | Every generator is an iterator                                                                |


## Common Mistake!!!!!!

``range()`` is not an iterator!

#### Key Differences Between Range and Iterator

| **Feature**           | **Range Object**                         | **Iterator Object**                   |
|------------------------|------------------------------------------|---------------------------------------|
| **Reusability**       | ✅  | ❌    |
| **Indexing**          | ✅  | ❌    |
| **Creation**          | Built-in using `range(start, stop)`.     | Created using `iter()` or custom classes. |
| **Lazy Evaluation**   | ✅  | ✅    |
| **Memory Efficiency** | Efficient, stores `start`, `stop`, `step`. | Efficient, processes one element at a time. |
| **Custom Behavior**   | Fixed behavior for numeric sequences.    | Customizable behavior with classes.   |


# Generators, Iterators, and Asynchronous Programming

Clean Code Github: https://github.com/PacktPublishing/Clean-Code-in-Python-Second-Edition/tree/main

## A first look at generators

Create dummy CSV data set

In [43]:
# <purchase_date>, <price>

In [42]:
"""Helper to generate test data."""
import os


PURCHASES_FILE = os.path.join(os.getcwd(), "purchases.csv")

def create_purchases_file(filename, entries=1_000_000):
    if os.path.exists(PURCHASES_FILE):
        return

    with open(filename, "w+") as f:
        for i in range(entries):
            line = f"2018-01-01,{i}\n"
            f.write(line)


if __name__ == "__main__":
    create_purchases_file(PURCHASES_FILE)

In [6]:
import logging

logging.basicConfig(level=logging.INFO, format="%(message)s")
logger = logging.getLogger(__name__)

In [44]:
class PurchasesStats:
    def __init__(self, purchases):
        self.purchases = iter(purchases)
        self.min_price: float = None
        self.max_price: float = None
        self._total_purchases_price: float = 0.0
        self._total_purchases = 0
        self._initialize()

    def _initialize(self):
        try:
            first_value = next(self.purchases)
        except StopIteration:
            raise ValueError("no values provided")

        self.min_price = self.max_price = first_value
        self._update_avg(first_value)

    def process(self):
        for purchase_value in self.purchases:
            self._update_min(purchase_value)
            self._update_max(purchase_value)
            self._update_avg(purchase_value)
        return self

    def _update_min(self, new_value: float):
        if new_value < self.min_price:
            self.min_price = new_value

    def _update_max(self, new_value: float):
        if new_value > self.max_price:
            self.max_price = new_value

    def _update_avg(self, new_value: float):
        self._total_purchases_price += new_value
        self._total_purchases += 1

    @property
    def avg_price(self):
        return self._total_purchases_price / self._total_purchases
    
    def __str__(self):
        return (
            f"{self.__class__.__name__}({self.min_price}, "
            f"{self.max_price}, {self.avg_price})"
        )

<span style='color: red'>Wrong approach:</span>

In [49]:
def load_purchases(filename):
    purchases = []
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            purchases.append(float(price_raw))

    return purchases

<span style='color: lightgreen'>Correct approach:</span>

In [50]:
def load_purchases(filename):
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            yield float(price_raw)

In [ ]:
purchases = load_purchases(PURCHASES_FILE)
stats = PurchasesStats(purchases).process()
logger.info("Results: %s", stats)

``yield`` keyword makes the generator object:

In [ ]:
load_purchases("file")

## Generator expressions

In [ ]:
[x**2 for x in range(10)]

In [ ]:
(x**2 for x in range(10))

Generator expressions can also be passed directly to functions that work with iterables, such as sum() and max():

In [ ]:
sum(x**2 for x in range(10))

In [ ]:
min(x**2 for x in range(10))

In [ ]:
max(x**2 for x in range(10))

<span style='color: lightgreen'>Attention!</span> Always pass a generator expression, instead of a list comprehension, to functions that expect iterables, such as min(), max(), and sum(). This is more efficient and Pythonic.

<span style='color: lightgreen'>Note:</span> Remember that generators are exhausted after they're iterated over, because they don't hold all the data in memory.

## Iterating idiomatically

### Idioms for iteration

We are already familiar with the built-in enumerate() function:

In [ ]:
list(enumerate("abcdef"))

We want an object that can produce a sequence
of numbers, from a starting one, without any limits; one that can
simply create an infinite sequence.

An object as simple as the following one can do the trick. Every time we call this
object, we get the next number of the sequence ad infinitum:

In [104]:
class NumberSequence:
    def __init__(self, start=0):
        self.current = start
        
    def next(self):
        current = self.current
        self.current += 1
        return current

In [86]:
seq = NumberSequence()

In [ ]:
seq.next() # Run as many as you like!

In [88]:
seq = NumberSequence(10) # Start from 10

In [ ]:
seq.next() # Run as many as you like!

In [ ]:
# Syntax reminder
list(zip([1,2,3,4,5], "abcdef"))

But the problem is 🫠:

In [ ]:
list(zip(NumberSequence(), "abcdef"))
# TypeError: zip argument #1 must support iteration

The problem lies in the fact that ``NumberSequence`` does not support iteration.

Fix:

In [118]:
class NumberSequence:
    def __init__(self, start=0):
        self.current = start

    # Change name() to __name__()
    def __next__(self):
        current = self.current
        self.current += 1
        return current
    
    # Add __iter__()
    def __iter__(self):
        return self

In [ ]:
list(zip(NumberSequence(), "abcdef"))

### The ``next()`` function

The ``next()`` built-in function will advance the iterable to its next element and
return it:

In [123]:
word = iter("hello")

In [ ]:
next(word)

If the iterator does not have more elements to produce, the StopIteration exception
is raised:

In [ ]:
word = iter("hello")
print(next(word))
print(next(word))
print(next(word))
print(next(word))
print(next(word))
print(next(word))

This exception signals that the iteration is over and that there are no more elements
to consume.

Handle with the by ``default value`` :

In [ ]:
next(word, "default value")

<span style='color: lightgreen'>Note:</span> It is advisable to use the default value most of the time, to avoid having exceptions
at runtime in our programs. If we are absolutely sure that the iterator we're dealing
with cannot be empty, it's still better to be use default value.

### Using a generator

The previous code can be simplified significantly by simply using a generator.

In [130]:
def sequence(start=0):
    while True:
        yield start
        start += 1

<strong>Because it is a generator</strong>, it's perfectly fine to create
an infinite loop like this

In [131]:
seq = sequence(10)

In [ ]:
next(seq) # Run as many times as you wish!

In [ ]:
list(zip(sequence(), "abcdef"))

## Simplifying code through iterators

### Nested loops

<span style='color: red'>Wrong approach:</span>

In [134]:
def search_nested_bad(array, desired_value):
    """Example of an iteration in a nested loop."""
    coords = None
    for i, row in enumerate(array):
        for j, cell in enumerate(row):
            if cell == desired_value:
                coords = (i, j)
                break

        if coords is not None:
            break

    if coords is None:
        raise ValueError(f"{desired_value} not found")

    logger.info("value %r found at [%i, %i]", desired_value, *coords)
    return coords

<span style='color: lightgreen'>Correct approach:</span>

In [135]:
def _iterate_array2d(array2d):
    for i, row in enumerate(array2d):
        for j, cell in enumerate(row):
            yield (i, j), cell


def search_nested(array, desired_value):
    """"Searching in multiple dimensions with a single loop."""
    try:
        coord = next(
            coord
            for (coord, cell) in _iterate_array2d(array)
            if cell == desired_value
        )
    except StopIteration as e:
        raise ValueError(f"{desired_value} not found") from e

    logger.info("value %r found at [%i, %i]", desired_value, *coord)
    return coord

<h3>Multi Processing</h3>

<img src="./01. Multi_Processing.png"/>

<h3>Threading</h3>

<img src="./02. Threading.png"/>

<h3>Coroutine</h3>

<img src="./03. Coroutine.png"/>

## Coroutines

The idea of a coroutine is to have a function, whose execution can be suspended
at a given point in time, to be later resumed.

## The methods of the generator interface

### ``close()``

When calling this method, the generator will receive the ``GeneratorExit`` exception.
If it's not handled, then the generator will finish without producing any more values,
and its iteration will stop.

In [1]:
class DBHandler:
    """Simulate reading from the database by pages."""

    def __init__(self, db):
        self.db = db
        self.is_closed = False

    def read_n_records(self, limit):
        return [(i, f"row {i}") for i in range(limit)]

    def close(self):
        logger.info("closing connection to database %r", self.db)
        self.is_closed = True

In [2]:
def stream_db_records(db_handler):
    try:
        while True:
            yield db_handler.read_n_records(10)
    except GeneratorExit:
        db_handler.close()

In [3]:
streamer = stream_db_records(DBHandler("testdb"))

In [ ]:
next(streamer)

At each call to the generator, it will return 10 rows obtained from the database
handler, but when we decide to explicitly finish the iteration and call close(),
we also want to close the connection to the database

In [ ]:
streamer.close()

<span style='color: lightgreen'>Note:</span> Use the ``close()`` method on generators to perform finishing-up
tasks when needed.

### ``throw(ex_type[, ex_value[, ex_traceback]])``

This method will throw the exception at the line where the generator is currently
suspended. If the generator handles the exception that was sent, the code in that
particular except clause will be called; otherwise, the exception will propagate to
the caller.

In [8]:
class CustomException(Exception):
    """An exception of the domain model."""

In [9]:
def stream_data(db_handler):
    while True:
        try:
            yield db_handler.read_n_records(10)
        except CustomException as e:
            logger.info("controlled error %r, continuing", e)
        except Exception as e:
            logger.info("unhandled error %r, stopping", e)
            db_handler.close()
            break

In [10]:
streamer = stream_data(DBHandler("testdb"))

In [ ]:
next(streamer)

In [ ]:
streamer.throw(CustomException)

In [ ]:
streamer.throw(RuntimeError)

### ``send(value)``

You can use ``send(value)`` to pass a value to the generator, which resumes execution from the yield statement.

In [14]:
def coro():
    print("Coroutine started")
    y = yield  # Waits for a value to be sent
    print(f"Received: {y}")

In [18]:
# Create the generator
gen = coro()

In [ ]:
# Prime the generator (start it up to the first yield)
next(gen)  # or gen.send(None)

In [ ]:
# Send a value to the generator
gen.send(42)

We would like to parametrize that number (10) so that we can change it throughout
different calls. Unfortunately, the next() function does not provide us with options
for that. But luckily, we have ``send()``:

In [23]:
def stream_db_records(db_handler):
    retrieved_data = None
    previous_page_size = 10
    try:
        while True:
            page_size = yield retrieved_data
            if page_size is None:
                page_size = previous_page_size

            previous_page_size = page_size

            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

In [27]:
streamer = stream_db_records(DBHandler("testdb"))

In [29]:
next(streamer)

In [ ]:
streamer.send(15)

Sending values to the coroutine only works when this one is suspended at a yield
statement, waiting for something to produce. For this to happen, the coroutine will
have to be advanced to that status. The only way to do this is by calling next() on it.
This means that before sending anything to the coroutine, this has to be advanced at
least once via the next() method. Failure to do so will result in an exception:

In [33]:
# Create the generator
gen = coro()

In [ ]:
# Send a value to the generator
gen.send(42)

<span style='color: lightgreen'>Note:</span> Always remember to advance a coroutine by calling next() before
sending any values to it.

More Pythonic:

In [35]:
def stream_db_records(db_handler):
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

In [ ]:
streamer = stream_db_records(DBHandler('testdb'))

In [37]:
next(streamer)

In [ ]:
streamer.send(5)

Get rid of ``next()`` function at before ``send()``:

In [39]:
def prepare_coroutine(coroutine):
    def wrapped(*args, **kwargs):
        advanced_coroutine = coroutine(*args, **kwargs)
        next(advanced_coroutine)
        return advanced_coroutine

    return wrapped

In [40]:
@prepare_coroutine
def auto_stream_db_records(db_handler):
    """This coroutine is automatically advanced so it doesn't need the first
    next() call.
    """
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

In [ ]:
streamer = auto_stream_db_records(DBHandler('testdb'))

In [ ]:
streamer.send(5)

# References

- Clean Code
- [Geeks for Geeks](https://www.geeksforgeeks.org/)
- [Stack Overflow](https://stackoverflow.com/)
- [Quera](https://quera.org/blog/coroutine-in-python/)